In [12]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

Interacting with the MLflow tracking server
The MlflowClient object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions.
  
To instantiate it we need to pass a tracking URI and/or a registry URI

In [2]:
client = MlflowClient(MLFLOW_TRACKING_URI)
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [3]:
client.create_experiment("my-cool-experiment")

'2'

In [10]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6.8",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"],
)

In [11]:
for run in runs:
    print(f"Run ID: {run.info.run_id}", f"RMSE: {run.data.metrics['rmse']}")

Run ID: 2bc681d8744e4639ad883c9e10cb544c RMSE: 6.396623670205836
Run ID: 7bd6482d897e40448bf187dc17e901f3 RMSE: 6.4167028006893165
Run ID: 1f6b61b2a90949bca23fdc5acbc0cbc3 RMSE: 6.422102873543081
Run ID: ecf6dbc7209042318cc19a4d6a8ac2e9 RMSE: 6.513756891884589
Run ID: 06ddfb8faf54444cace30c5d84cb5294 RMSE: 6.523179033040928


## Interacting with the Model Registry

In this section We will use the MlflowClient instance to:

- Register a new version for the experiment nyc-taxi-regressor
- Retrieve the latests versions of the model nyc-taxi-regressor and check that a new version 4 was created.

Transition the version 4 to "Staging" and adding annotations to it.

In [15]:
run_id = "ecf6dbc7209042318cc19a4d6a8ac2e9"
model_uri = f"runs:/{run_id}/model"

mlflow.register_model(model_uri, "nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/09/30 11:26:37 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1664555197772, current_stage='None', description=None, last_updated_timestamp=1664555197772, name='nyc-taxi-regressor', run_id='ecf6dbc7209042318cc19a4d6a8ac2e9', run_link=None, source='./mlruns/1/ecf6dbc7209042318cc19a4d6a8ac2e9/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [24]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")


version: 1, stage: Production
version: 3, stage: Staging
version: 4, stage: None


In [31]:
model_version = 1
new_stage = "Staging"

client.transition_model_version_stage(
    name = model_name,
    version = model_version,
    stage = new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1664554354734, current_stage='Staging', description='This is a model version 1 was transitioned to Staging on 2022-09-30 11:37:20', last_updated_timestamp=1664555878244, name='nyc-taxi-regressor', run_id='ecf6dbc7209042318cc19a4d6a8ac2e9', run_link=None, source='./mlruns/1/ecf6dbc7209042318cc19a4d6a8ac2e9/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [28]:
# add description
new_stage = "Staging"
from datetime import datetime

date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
client.update_model_version(
    name = model_name,
    version = model_version,
    description = f"This is a model version {model_version} was transitioned to {new_stage} on {date}",
)

<ModelVersion: creation_timestamp=1664554354734, current_stage='Staging', description='This is a model version 1 was transitioned to Staging on 2022-09-30 11:37:20', last_updated_timestamp=1664555840595, name='nyc-taxi-regressor', run_id='ecf6dbc7209042318cc19a4d6a8ac2e9', run_link=None, source='./mlruns/1/ecf6dbc7209042318cc19a4d6a8ac2e9/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

## compare performance and transition to "Production"

In [33]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [34]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [37]:
import pickle

with open("models/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [38]:
X_test = preprocess(df, dv)

In [40]:
target = "duration"
y_test = df[target].values

In [42]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 78.7 ms, sys: 32.5 ms, total: 111 ms
Wall time: 172 ms


{'rmse': 6.490821331307093}

In [43]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

[11:47:19] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 5.1 s, sys: 103 ms, total: 5.2 s
Wall time: 1.05 s


{'rmse': 6.482732906034752}

In [45]:
client.transition_model_version_stage(
    name = model_name,
    version = 4,
    stage = "Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1664555536365, current_stage='Production', description='', last_updated_timestamp=1664556642384, name='nyc-taxi-regressor', run_id='7d518b2bc56342319a1988fa0f37594e', run_link='', source='./mlruns/1/7d518b2bc56342319a1988fa0f37594e/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=4>